# 1) Questões de Negócio
**Nome da Empresa**: House Rocket<br>
**Modelo de Negócio**: Compra e venda de imóveis, obtendo lucro pela revenda.<br>
**Problemas de Negócio**: 
1. Quais os imóveis que a House Rocket deveria comprar e por qual preço?
2. Uma vez comprados, quando será a melhor época para revender e por qual preço?
3. Valide as seguintes *hipóteses de negócio*:
   1. Imóveis que possuem vista para água, são 20% mais caros, na média.
   2. Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.
   3. Imóveis sem porão - possuem área total (sqrt_lot) - são 40% maiores do que os imóveis com porão.
   4. O crescimento do preço dos imóveis YoY (Year over Year) é de 10%.
   5. Imóveis com 3 banheiros tem um crescimento de MoM (Month over Month) de 15%.
   6. Imóveis com mais números de quarto são em média 10% mais caros do que outros imóveis com 1 unidade de quartos a menos, em média.
   7. Imóveis antigos construidos antes de 1970, que nunca foram renovados, são 40% mais baratos em média.
   8. Imóveis que foram renovados recentemente são 19% mais caros, na mediana.
   9.  Imóveis em más condições mas possuem vista para o mar, são 10% mais caras, na mediana.
   10. O crescimento MoM (Month over Month) do preço das propriedades é de 1%, na média.

# 2) Planejamento da Solução
### Produto Final
- Dashboard disponível online 24/7, contendo:
  - 2 tabelas referentes a recomendações de imóveis e preços de compra e revenda.
  - Visualiações correspondendo a validação de cada uma das hipóteses de negócio.
### Planejamento das Ferramentas
- Python 3.10.*
- Pandas, Numpy
- Matplotlib e Seaborn
- Streamlit
- Cloud Heroku
### Processo
##### 1) Quais os imóveis que a House Rocket deveria comprar e por qual preço?
1. Agrupar os imóveis por região;
2. Aplicar mediana do preço de cada região;
3. Selecionar imóveis que estejam abaixo do preço mediano de sua região e que estejam em boas condições;
##### 2) Uma vez comprados, quando será a melhor época para revender e por qual preço?
1. Agrupar os imóveis por região e por sazionalidade (verão e inverno);
2. Calcular preço mediano de cada grupo;
3. Selecionar aqueles imóveis que estejam em boas condições e que possuam seu preço abaixo da mediana de seu grupo;
4. 
##### 3) Valide as seguintes *hipóteses de negócio*:
1. **Imóveis que possuem vista para água, são 20% mais caros, na média.**
2. **Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.**
3. **Imóveis sem porão - possuem área total (sqrt_lot) - são 40% maiores do que os imóveis com porão.**
4. **O crescimento do preço dos imóveis YoY (Year over Year) é de 10%.**
5. **Imóveis com 3 banheiros tem um crescimento de MoM (Month over Month) de 15%.**
6. **Imóveis com mais números de quarto são em média 10% mais caros do que outros imóveis com 1 unidade de quartos a menos, em média.**
7. **Imóveis antigos construidos antes de 1970, que nunca foram renovados, são 40% mais baratos em média.**
8. **Imóveis que foram renovados recentemente são 19% mais caros, na mediana.**
9. **Imóveis em más condições mas possuem vista para o mar, são 10% mais caras, na mediana.**
10. **O crescimento MoM (Month over Month) do preço das propriedades é de 1%, na média.**

# 3) Coleta e Limpeza dos Dados

# 4) Análise Exploratória dos Dados